In [1]:
##first we see data cleaning using nltk 
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV as GSC
from sklearn import svm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [3]:
df=pd.read_csv("twitter training.csv")
df.head()


,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
text=list(df['text'])
text1=list(df['airline_sentiment'])

In [5]:
documents=[]
for i in range(len(text)):
    documents.append((text[i].split(" "),text1[i]))
documents[5:9]    

[(['@united',
   'Why',
   'did',
   'you',
   'load',
   'us',
   'in',
   'this',
   'flying',
   'sardine',
   'can',
   'if',
   'you',
   'knew',
   'the',
   'pilots',
   'were',
   '2',
   'hours',
   'Late',
   'Flight??',
   '#incompetent',
   'beyond',
   'belief'],
  'negative'),
 (['@JetBlue',
   'that',
   'is',
   'a',
   'stock',
   'response.',
   'Delays',
   'not',
   'as',
   'frustrating',
   'as',
   'poor',
   'cust',
   'serv',
   '&amp;',
   'being',
   'told',
   'by',
   '3',
   'ppl',
   'to',
   'wait',
   '&amp;',
   "they'd",
   'come',
   'back',
   'but',
   'did',
   'not.'],
  'negative'),
 (['@JetBlue',
   "That'd",
   'be',
   'nice!',
   'Hoping',
   'to',
   'rack',
   'up',
   'enough',
   'miles',
   'to',
   'take',
   'a',
   'trip',
   'to',
   'Seattle',
   'and',
   'enjoy',
   'a',
   'perfect',
   'latte',
   'in',
   'the',
   'city',
   'of',
   'coffee.'],
  'positive'),
 (['@united',
   'frankly',
   'worse',
   'customer',
   'service

In [6]:
df1=pd.read_csv("twitter testing.csv")
df1.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [7]:
text_testing= list(df1['text'])
text_testing[:3]

["@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can't reach arpt for AA2450. Wat 2 do?",
 '@AmericanAir after all, the plane didn’t land in identical or worse) conditions at GRK according to METARs.',
 "@SouthwestAir can't believe how many paying customers you left high and dry with no reason for flight Cancelled Flightlations Monday out of BDL! Wow."]

In [8]:
test_documents=[]
for i in range(len(text_testing)):
    test_documents.append(text_testing[i].split(" "))
test_documents[:2]  

[['@AmericanAir',
  'In',
  'car',
  'gng',
  'to',
  'DFW.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr.',
  "Can't",
  'reach',
  'arpt',
  'for',
  'AA2450.',
  'Wat',
  '2',
  'do?'],
 ['@AmericanAir',
  'after',
  'all,',
  'the',
  'plane',
  'didn’t',
  'land',
  'in',
  'identical',
  'or',
  'worse)',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs.']]

In [9]:
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

In [10]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith ('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
        

In [11]:
from nltk import pos_tag
## adding stop words
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
## adding punctuations
import string
punctuations = list(string.punctuation)
stop.update(punctuations)
stop,string.punctuation

({'!',
  '"',
  '#',
  '$',
  '%',
  '&',
  "'",
  '(',
  ')',
  '*',
  '+',
  ',',
  '-',
  '.',
  '/',
  ':',
  ';',
  '<',
  '=',
  '>',
  '?',
  '@',
  '[',
  '\\',
  ']',
  '^',
  '_',
  '`',
  'a',
  'about',
  'above',
  'after',
  'again',
  'against',
  'ain',
  'all',
  'am',
  'an',
  'and',
  'any',
  'are',
  'aren',
  "aren't",
  'as',
  'at',
  'be',
  'because',
  'been',
  'before',
  'being',
  'below',
  'between',
  'both',
  'but',
  'by',
  'can',
  'couldn',
  "couldn't",
  'd',
  'did',
  'didn',
  "didn't",
  'do',
  'does',
  'doesn',
  "doesn't",
  'doing',
  'don',
  "don't",
  'down',
  'during',
  'each',
  'few',
  'for',
  'from',
  'further',
  'had',
  'hadn',
  "hadn't",
  'has',
  'hasn',
  "hasn't",
  'have',
  'haven',
  "haven't",
  'having',
  'he',
  'her',
  'here',
  'hers',
  'herself',
  'him',
  'himself',
  'his',
  'how',
  'i',
  'if',
  'in',
  'into',
  'is',
  'isn',
  "isn't",
  'it',
  "it's",
  'its',
  'itself',
  'just',
  'll',


In [12]:
### cleaning and lemmatization
def clean_text(words):
    op_words=[]
    for w in words:
        if w.lower() not in stop and w.isalpha() and len(w)>2 and not w.isnumeric():
            pos = pos_tag([w])
            clean_word = lem.lemmatize(w,pos = get_simple_pos(pos[0][1]))
            op_words.append(clean_word.lower())
    return op_words        

In [13]:
documents = [(clean_text(document),category)for document,category in documents]

In [14]:
test_documents = [(clean_text(doc)) for doc in test_documents]

In [15]:
documents[:5]

[(['schedule',
   'day',
   'sure',
   'even',
   'flight',
   'one',
   'cancelled',
   'flightled'],
  'negative'),
 (['see', 'worker', 'time', 'time', 'go', 'beyond', 'love', 'fly', 'thank'],
  'positive'),
 (['flew', 'ord', 'miami', 'back', 'great', 'service', 'thanks'], 'positive'),
 (['horse', 'radish'], 'negative'),
 (['flight', 'ord', 'delayed', 'air', 'force', 'last', 'flight', 'sbn', 'min'],
  'negative')]

In [16]:
test_documents[:5]

[['car', 'gng', 'pulled', 'ago', 'icy', 'since', 'reach', 'arpt', 'wat'],
 ['plane', 'land', 'identical', 'condition', 'grk', 'accord'],
 ['believe',
  'many',
  'pay',
  'customer',
  'left',
  'high',
  'dry',
  'reason',
  'flight',
  'cancelled',
  'flightlations',
  'monday'],
 ['legitimately',
  'say',
  'would',
  'rather',
  'driven',
  'cross',
  'country',
  'flown'],
 ['still', 'response', 'great', 'job']]

In [17]:
## count_vectorizer fnction
from sklearn.feature_extraction.text import CountVectorizer as cv


In [18]:
categories = [category for doc,category in documents]
text_documents=[" ".join(doc) for doc,category in documents]
test_documents=[" ".join(doc) for doc in test_documents]
text_documents[:2],categories[:2]

(['schedule day sure even flight one cancelled flightled',
  'see worker time time go beyond love fly thank'],
 ['negative', 'positive'])

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train,y_test=train_test_split(text_documents,categories,test_size=0.3)
x_train[:2]

['dmed back', 'minute still talk one person phone gonna die talk anyone']

In [21]:
count_vec = cv(max_features=6500,max_df=0.92,stop_words=stop)
x_train_feat = count_vec.fit_transform(x_train)
x_train_feat = x_train_feat.todense()

In [22]:
count_vec.get_feature_names()

['aaaand',
 'aadvantage',
 'aaron',
 'abandon',
 'abassinet',
 'abc',
 'abduct',
 'abi',
 'ability',
 'able',
 'aboard',
 'aboout',
 'abq',
 'abroad',
 'absolute',
 'absolutely',
 'absoulutely',
 'absurd',
 'absurdly',
 'abt',
 'abundance',
 'abuse',
 'accept',
 'acceptable',
 'accepted',
 'acces',
 'access',
 'accessible',
 'accident',
 'accidentally',
 'accidents',
 'accommodate',
 'accommodation',
 'accompaniment',
 'accompany',
 'accomplish',
 'accord',
 'according',
 'accordingly',
 'account',
 'accountability',
 'accrue',
 'acct',
 'accts',
 'accurate',
 'accurately',
 'accuse',
 'acknowledge',
 'acknowledgment',
 'acosta',
 'acquire',
 'across',
 'act',
 'action',
 'active',
 'actual',
 'actually',
 'actualy',
 'acu',
 'ad',
 'adapt',
 'add',
 'adding',
 'addition',
 'additional',
 'additonal',
 'addr',
 'address',
 'adds',
 'addtl',
 'adjacent',
 'adjustment',
 'admiral',
 'admirals',
 'admit',
 'adore',
 'adult',
 'adv',
 'advan',
 'advance',
 'advantage',
 'advertise',
 'adve

In [23]:
x_test_feat = count_vec.transform(test_documents)
x_test_feat1= count_vec.transform(x_test)

In [24]:
x_test_feat.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rfc = RandomForestClassifier()
rfc.fit(x_train_feat,y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [27]:
rfc.score(x_train_feat,y_train)# bad on testing

0.9767109029404112

In [28]:
from sklearn.naive_bayes import MultinomialNB as MNB
clf=MNB(alpha=1)
clf.fit(x_train_feat,y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [29]:
clf.score(x_train_feat,y_train)


0.8049700754618787

In [30]:
clf.score(x_test_feat1,y_test)

0.7234365513054037

In [31]:
#clf2=svm.SVC()

#clf2.fit(x_train_feat,y_train)

In [32]:
#clf2.score(x_train_feat,y_train)

In [33]:
as2=clf.predict(x_test_feat)

In [34]:
as1=rfc.predict(x_test_feat)

In [35]:
#as3=clf2.predict(x_test_feat)

In [36]:
as1

array(['negative', 'neutral', 'negative', ..., 'neutral', 'positive',
       'negative'], dtype='<U8')

In [37]:
as2

array(['negative', 'neutral', 'negative', ..., 'positive', 'positive',
       'neutral'], dtype='<U8')

In [38]:
np.savetxt("sentiment.csv",as2, fmt="%s")